Want a way to save/load our data.
Can use HDF5 for maximum portability or JLD if we are comfortable staying in the Julia ecosystem.
Some patterns we could use:

1. Have every worker send their data to the driver, and have it write one file
2. Have each worker write its own data file, then collate them at the end using some script
3. Have a group of workers dedicated to collecting data and collating

3 is intermediate between 1 and 2. Each one has advantages and disadvantages. 3 will let us show off some cool Julia features like `WorkerPool` and it's also the most difficult to get right, so let's do that!

It's a good idea to keep track of what version of the code we used to generate the data. 